In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

# Exercise 3

## Contents
- Gradient Descent
- Backpropagation
- Backpropagation in PyTorch - Coding Example

## Gradient Descent

**Cost function:** tells us how well did our network perform on examples given chosen weights and biases. 
- But how did that error come to be? Which weights (and biases) did we set correctly? Which ones not? What way do we need to alter them?

**Remember from calculus:** gradient of a function tells us direction of steepest ascent!
- Therefore: moving in opposite direction (steepest descent) will minimize our function the most!

**Goal of gradient descent:** decrease the cost function.

- Update parameters ($\theta$) in direction of steepest descent (will lead you to a local minimum)
- **Gradient vector $\nabla E(\theta(k))$:** what direction do we adjust each parameter?
- **Learning rate $\tau$:** how big are our adjustments?

**Anology**: walking down a mountain taking small steps into the direction of steepest descent instead of large ones until you reach a valley

**Formula:** $ \theta(k+1) = \theta(k) - \tau \nabla E(\theta(k)) $

For gradient descent we need thus need the partial derivative of our cost function with respect to **each weight and bias**!

- **Gradient vector** will tell us **which direction to adjust each weight**, i.e. "turn the knobs of our network"!

- **Magnitude of each entry** in the gradient vector tells you how much it affected the outcome of the cost function (therefore, adjusting it in the direction of steepest descent (negative gradient) will have larger impact on reaching local minimum)
    - For example: z = 2x + y
    - $ \frac{dz}{dx} = 2 $ and $ \frac{dz}{dy} = 1 $, i.e. x contributes twice as much as y to the outcome of z

## Backpropagation

**Computing the gradient of a deeply nested function**
- **Essentially:** applying chain rule over and over again
- Chain rule: $E = f(g(x)) \rightarrow \nabla E(x) = \nabla g(x) * \nabla f(g(x))$

**Unravels our deeply nested functions in a backward manner!**

**Important observation:** During gradient calculation in a compute graph, each node is just interested in its own input and output.
- Only need to compute local gradient with respect to its inputs
- Then multiplied with gradient that has been backpropagated to this node

<img src="backprop_idea.jpg" alt="Drawing" style="width: 60%;"/>

What if we are dealing with matrices? Same thing, gradients become matrices (transposed jacobians).
- **Note:** Important to keep the order on the backward pass! $A @ B \neq  B @ A$ 

<img src="backprop_idea_matrix.jpg" alt="Drawing" style="width: 60%;"/>

This is a lot to digest! Take your time and practice your skills by drawing simple compute graphs and calculating the partial derivatives.

I recommend the videos of 3Blue1Brown on this topic:
- https://youtu.be/IHZwWFHWa-w
- https://youtu.be/Ilg3gGewQ5U
- https://youtu.be/tIeHLnjs5U8

## Backpropagation in PyTorch - Coding Example


**Tensors**: data structure used in PyTorch (uses taping for remembering computations)

- **Attributes:**
    - ```data```: our data stored in the Tensor (e.g. input features) 
    - ```requires_grad```: want to compute gradient with respect to Tensor at some point. 
    - ```grad```: If ```requires_grad``` is ```True``` initialised as zero-array with shape of ```data``` (every entry has a gradient associated with it).
    - ```grad_fn```: Function which created the tensor (e.g. result of an addition).
    - ```num_forward```: tracks how many (forward) functions were applied on top of the tensor.

- **Functions:**
    - ```backward```: how is the backward pass computed for this Tensor? Essentially two steps: add up incoming gradient to gradient to gradient of Tensor and call backward on gradient function (continue the backward pass and send along local gradient)
    - ```zero_grad```: resets the gradients of the Tensor to all zeros again
    - ```__add__```: the sample operation we are overwriting in this exercise (calls the Add() function's forward function)

In [2]:
class Tensor():
    def __init__(self, data, requires_grad=False):
        self.data = np.array(data)
        self.requires_grad = requires_grad
        if self.requires_grad:
            self.grad = np.zeros_like(self.data)
        self.grad_fn = None
        self.num_forward = -1
    
    def backward(self, dLdt=1.):
        if self.requires_grad:
            self.grad += dLdt
            if self.num_forward > 0:
                self.num_forward -= 1
            elif self.grad_fn is not None:
                self.grad_fn.backward(self.grad)
                del self.grad_fn
    
    def zero_grad(self):
        self.grad = np.zeros_like(self.data)
    
    def __add__(self, other):
        return Add().forward(self, other)

In the exercise you will implement child classes of the parent class `Function()` (resembles basic mathmatical computations that can be performed on `Tensors()`)
- **Functions:**
    - `forward()`: Given an input, how is the mathmatical operation performed and what is the output?
    - `backward()`: Given an incoming gradient, how is the local gradient computed and propagated backwards to each input parameter? 

In [3]:
class Function():
    def forward(self, *tensors):
        for t in tensors:
            t.num_forward += 1
    
    def backward(self, dLdf):
        pass

**Addition Example:**

- `forward()`: given input tensor `t1` and `t2`:
    - save references to tensors involved in computation
    - output is new tensor with  `data = t1.data + t2.data`
    - if at least one input requires gradient, `requires_grad = True` of output tensor
    - `grad_fn` of output is set to `Add()` (i.e. `self`)

- `backward()`: given incoming gradient `dLdf`:
    - Compute local gradient with respect to each input (`t1` and `t2`)
    - **Idea:** send local gradient of input + incoming gradient backwards to respective input

In [4]:
class Add(Function):
    def forward(self, t1, t2):
        super().forward(t1, t2)
        self.t1 = t1
        self.t2 = t2
        requires_grad = t1.requires_grad or t2.requires_grad
        t = Tensor(t1.data+t2.data, requires_grad=requires_grad)
        t.grad_fn = self
        return t
    
    def backward(self, dLdf):
        # gradient with respect to t1
        dfdt1 = 1
        grad_t1 = dfdt1 * dLdf
        self.t1.backward(grad_t1)
        
        # gradient with respect to t2
        dfdt2 = 1
        grad_t2 = dfdt2 * dLdf
        self.t2.backward(grad_t2)

**Exercise:** what are the partial derivatives with respect to each parameter for this computation graph?

$$ a + b = c $$

$$ a + c = d $$

$$ \frac{dd}{dd} = \text{  ?  ;}  \frac{dd}{dc} = \text{  ?  ;}  \frac{dd}{db} = \text{  ?  ;}  \frac{dd}{da} = \text{  ?  } $$

<img src="compute_graph.png" alt="Drawing" style="width: 30%;"/>

**Solution using backpropagation:**
- Start at end of compute graph, i.e. at $d$. 
    - Compute the local derivatives with respect to the inputs (i.e. $c$ and $a$)
        - Thus compute: $\frac{dd}{dc} = 1$ and $\frac{dd}{da} = 1$
    - Since there was no incoming gradient, we default it to 1
        - Equivalent to $\frac{dd}{dd} = 1$
- From $d$ send back to each input the local gradient with respect to said input times the incoming gradient
     - Send back to $c$: $\frac{dd}{dc} * 1$
     - Send back to $a$: $\frac{dd}{da} * 1$
- At $c$: 
    - Add up the incoming gradients towards the gradient of $c$ (thus $\frac{dd}{db} = 1$)
    - Compute the local derivatives with respect to the inputs (i.e. $a$ and $b$).
        - Thus compute: $\frac{dc}{da} = 1$ and $\frac{dc}{db} = 1$
    - The incoming gradient from $d$ was 1
- From $c$:
    - Send back to a: $\frac{dc}{da} * 1$
    - Send back to b: $\frac{dc}{db} * 1$
- At $b$:
    - Add up the incoming gradients towards the gradient of $c$ (thus $\frac{dd}{db} = 1$)
    - Node has no inputs; backpropagation finishes
- At $a$:
    - Add up the incoming gradients towards the gradient of $a$ 
        - $a$ has two incoming gradients (from $d$ and $c$) (thus $\frac{dd}{da} = 1 + 1$)
    - Node has no inputs; backpropagation finishes

**Or directly written out using chain rule:**

$$ \frac{dd}{dd} = 1 $$

$$ \frac{dd}{dc} = 1$$

$$ \frac{dd}{db} = \frac{dd}{dc} * \frac{dc}{db} = 1 * 1 = 1 $$

$$ \frac{dd}{da} = \frac{dd}{da} + \frac{dd}{dc} * \frac{dc}{da}= 1 + 1 * 1 = 2 $$

In [5]:
a = Tensor(1., requires_grad=True)
b = Tensor(2., requires_grad=True)

c = a + b
d = c + a

Calling backward on d will now compute the gradient along the computation graph for each variable.

In [6]:
d.backward()

print(d.grad)
print(c.grad)
print(b.grad)
print(a.grad)

1.0
1.0
1.0
2.0


**Question:** can we call `backward()` again?

In [7]:
d.backward()

AttributeError: 'Tensor' object has no attribute 'grad_fn'

**Answer:** no, computation graph is deleted after the first gradient backpropagation as most of the times there is no use in keeping this graph alive. Therefore a new computation graph is built for every forward pass.

**Question:** what happens if we run the code below?

In [8]:
a = Tensor(1., requires_grad=True)
b = Tensor(2., requires_grad=True)

c = a + b
d = c + a

d.backward()

c = a + b
d = c + a

d.backward()

**Answer:** gradient accumulates! That is why we need to call `zero_grad()`to reset the gradient.

In [9]:
print(d.grad)
print(c.grad)
print(b.grad)
print(a.grad)

1.0
1.0
2.0
4.0
